# 重复100次实验  取平均值

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE                # 过抽样处理库SMOTE
from imblearn.under_sampling import RandomUnderSampler  # 欠抽样处理库RandomUnderSample
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA           #加载PCA算法包
from sklearn.metrics import mean_squared_error as mse
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_names = ['data.xlsx','AES_T400.xlsx','AES_T700.xlsx','AES_T800.xlsx','AES_T900.xlsx',
                  'AES_T1200.xlsx','AES_T1300.xlsx','AES_T1400.xlsx','AES_T2000.xlsx',]

def create_numerics(data):
    # Get nominal columns
    nominal_cols = data.select_dtypes(include='object').columns.tolist()

    # Turn nominal to numeric
    for nom in nominal_cols:
        enc = LabelEncoder()
        enc.fit(data[nom])
        data[nom] = enc.transform(data[nom])
    return data
    
def prepare_data(case = 0 , feature_choice = 0, data_id = 0, ispca = 0):

    data = pd.read_excel(data_names[data_id])
    data = data.dropna()

    data = create_numerics(data)
    data = shuffle(data)
    
    data = data.values
    y = data[:,-1]
    x = data[:,0:-1]
    if(ispca == 1):
        pca = PCA(n_components=0.9)
        pca.fit(x)
        x = pca.transform(x)
    
    if(feature_choice == 1):    #我们的  先验知识
        l=[0,4,8,12]
        x = data[:,l]
    elif(feature_choice == 2):  # 非均衡下 SHAP的结果  和   任何情况下 PI 的结果
        l=[0,1,3,4,8,12]  
        x = data[:,l]    
    elif(feature_choice == 3):  # 均衡下  SHAP 的结果  
        l=[0,4,8,10,12,15]
        x = data[:,l]    
        
    if(data_id == 0):    
        print("Original samples:",len(y))
        if(case ==1):
            model_smote = SMOTE(random_state=42)                      # 建立SMOTE模型对象
            x, y = model_smote.fit_sample(x, y)         # 输入数据做过抽样处理
        elif(case ==2):
            model_RandomUnderSample = RandomUnderSampler(random_state=42)   # 建立RandomUnderSampler模型对象
            x, y = model_RandomUnderSample.fit_sample(x, y)   # 输入数据做欠抽样处理
        elif(case ==3):
            oversample = BorderlineSMOTE(random_state=42)
            x, y = oversample.fit_resample(x, y)
        elif(case ==4):
            smt = SMOTETomek(random_state=42)
            x,y =  smt.fit_sample(x, y)
        elif(case ==5):
            smt = SMOTEENN(random_state=42)
            x,y =  smt.fit_sample(x, y)    
        print("New samples:",len(y))
        print(y.sum())
    scaler = MinMaxScaler(feature_range=(0, 1))
    x = scaler.fit_transform(x)
    return(x,y)
#     x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=ratio, random_state=1)
#     return(x_train, x_test, y_train, y_test)

In [3]:
def RF_test(rf,x_test, y_test):
    
    start = time.time()
    y_pred = rf.predict(x_test)
    end = time.time()
    
    time_    = 1000*(end - start)  # ms
    accuracy = 100 * accuracy_score(y_test, y_pred)
    f1score  = 100 * f1_score(y_test, y_pred, average='macro')
    
    total_test   = len(y_test)
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(total_test):
        if(y_test[i] == y_pred[i]   and y_pred[i]==1): #trojan
            tp += 1
        elif(y_test[i] == y_pred[i] and y_pred[i]==0): #normal
            tn += 1
        elif(y_test[i] != y_pred[i] and y_pred[i]==1): #normal
            fp += 1
        elif(y_test[i] != y_pred[i] and y_pred[i]==0): #normal
            fn += 1
    
    tpr         = tp/(tp+fn)
    tnr         = tn/(tn+fp)
    precision   = tp/(tp+fp)
    
    print("在数据集"+str(data_names[id_data]+"上的测试结果"))
    print("TIME：",time_)
    print("TPR ：",tpr)
    print("TNR ：",tnr)
    print("ACC ：",accuracy)
    print("PRE ：",precision)
    print("F1  ：" ,f1score)

In [4]:
def one_exp(rf, sampling, id_data, id_features, is_pca):

    # 测试集  不扩充， 但是要选维度   选 数据集编号 
    x_test, y_test = prepare_data(case = 0 , feature_choice = id_features, data_id = id_data, ispca = is_pca)
    RF_test(rf,x_test, y_test)

# 重复实验

In [5]:
# sampling cases: 0不处理    1 过采样Smote  2 随机欠采样      3 Borderline Smote  4 Smote+Toek 5 SmoteEEN

# 特征选择      ：0： 全选   1： 1 4 8 12   2：1 2 4 5 9 13   3：1 4 5 9 13 16

# 数据选择 data_names = ['data.xlsx','AES_T400.xlsx','AES_T700.xlsx','AES_T800.xlsx','AES_T900.xlsx',
# 'AES_T1200.xlsx','AES_T1300.xlsx','AES_T1400.xlsx','AES_T2000.xlsx',]
# 0 是训练集  其他 1-8 是测试集

# PCA处理       ：0 则不PCA  1  则PCA


if __name__ == '__main__':
    sampling = 5     # 过采样技术  建议 5
    id_features = 0  # 降维技术    建议 2
    is_pca = 0       # 是否PCA     建议 0 
    x_train, y_train = prepare_data(case = sampling , feature_choice = id_features, data_id = 0, ispca = is_pca)
    
    rf = RandomForestClassifier(n_estimators=5, max_depth=5, random_state=1)
    rf.fit(x_train, y_train)   

Original samples: 6915
New samples: 11942
5958


In [6]:
    id_data = 1
    one_exp(rf, sampling, id_data, id_features, is_pca)

在数据集AES_T400.xlsx上的测试结果
TIME： 1.9948482513427734
TPR ： 0.9871794871794872
TNR ： 1.0
ACC ： 99.98507685420086
PRE ： 1.0
F1  ： 99.67364491534298


In [7]:
    id_data = 2
    one_exp(rf, sampling, id_data, id_features, is_pca)

在数据集AES_T700.xlsx上的测试结果
TIME： 2.0225048065185547
TPR ： 1.0
TNR ： 1.0
ACC ： 100.0
PRE ： 1.0
F1  ： 100.0


In [8]:
    id_data = 3
    one_exp(rf, sampling, id_data, id_features, is_pca)

在数据集AES_T800.xlsx上的测试结果
TIME： 1.9948482513427734
TPR ： 1.0
TNR ： 1.0
ACC ： 100.0
PRE ： 1.0
F1  ： 100.0


In [9]:
    id_data = 4
    one_exp(rf, sampling, id_data, id_features, is_pca)

在数据集AES_T900.xlsx上的测试结果
TIME： 1.9946098327636719
TPR ： 0.9978354978354979
TNR ： 1.0
ACC ： 99.98601202965449
PRE ： 1.0
F1  ： 99.94209050131124


In [10]:
    id_data = 5
    one_exp(rf, sampling, id_data, id_features, is_pca)

在数据集AES_T1200.xlsx上的测试结果
TIME： 1.9946098327636719
TPR ： 0.9978354978354979
TNR ： 1.0
ACC ： 99.98601202965449
PRE ： 1.0
F1  ： 99.94209050131124


In [11]:
    id_data = 6
    one_exp(rf, sampling, id_data, id_features, is_pca)

在数据集AES_T1300.xlsx上的测试结果
TIME： 1.9650459289550781
TPR ： 1.0
TNR ： 1.0
ACC ： 100.0
PRE ： 1.0
F1  ： 100.0


In [12]:
    id_data = 7
    one_exp(rf, sampling, id_data, id_features, is_pca)

在数据集AES_T1400.xlsx上的测试结果
TIME： 1.9941329956054688
TPR ： 1.0
TNR ： 1.0
ACC ： 100.0
PRE ： 1.0
F1  ： 100.0


In [13]:
    id_data = 8
    one_exp(rf, sampling, id_data, id_features, is_pca)

在数据集AES_T2000.xlsx上的测试结果
TIME： 2.000093460083008
TPR ： 1.0
TNR ： 1.0
ACC ： 100.0
PRE ： 1.0
F1  ： 100.0
